In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [2]:
TEXT = """У топ-менеджера две основные функции. Первая — добиваться поставленных целей. Вторая — научить коллег добиваться поставленных целей.

***
Мы не любим ставить перед собой большие цели. Наш мозг, все наше естество противится этому. Почему? Ответ парадоксален, но точен. Потому что боимся, что они будут достигнуты! Да, да, именно поэтому.

***
Какая у человека будет цель, такая у него будет и жизнь.

***
Если у тебя нет стратегии — значит, у тебя нет бизнеса. Гендиректор, который не имеет цели хотя бы на пять лет, не может называть себя генеральным директором.

***
Учите свою компанию учиться. Новые знания способны дать бизнесу больше, чем миллионы инвестиций.

***
Люди любят, когда их хвалят. Для многих это важнее секса и денег. Потребность во внешнем признании сидит в нас так же глубоко, как волчья боязнь всего нового, что тоже неоднократно доказано психологическими экспериментами.

***
Чего бы ты ни достиг, ты всегда находишься в точке возможной погибели

***
Что бы там ни говорили про лихие 90-е, во все времена люди ценили честность и порядочность. И если придерживаться строгих правил и выполнять обещания, то риски можно свести к минимуму всегда и везде.

***
Улыбка генерального директора не элемент косметики для корпоративной культуры. Улыбка босса стоит миллионы в самом прямом смысле этого слова. Что делают сотрудники, когда руководитель приходит утром на предприятие? Смотрят: какое у шефа настроение. И не важно, карьерист ты или добросовестный трудяга. Это наш биологический инстинкт — постоянно сверять свои действия с сигналами вожака.

***
Надо приходить на работу и делиться, делиться, делиться хорошим настроением. Заряжать и заражать. Вот увидите, ваши сотрудники неизбежно передадут свое настроение клиентам. А клиенты — это наше всё.

***
Если к вам на работу придет устраиваться Билл Гейтс, хорошенько подумайте: нужен ли он вашей компании?

***
Человеку изначально даются лишь черты характера и склонность к определенного рода занятиям, которая диктуется набором генов. Все остальное — результат его упорства, воли, а также влияния людей, которые встретились на жизненном пути, и обстоятельств.

***
Ваша задача — не просто найти достойного человека, а выбрать достойного человека, наилучшим образом подходящего для конкретной задачи.

***
Все люди важны и эффективны. Бесполезных не бывает. Главное — найти подход, который поможет раскрыть их потенциал. А это наша, менеджерская задача.

***
Ищите своего покупателя и работайте только для него. Остальные придут сами.

***
Когда требования ваших покупателей грубо противоречат вашим принципам и ценностям, выбирать нужно принципы и ценности.

***
Бизнес говорит на языке цифр. А цифры — прежде всего ответственность. Поэтому цифры у нас никто не любит.

***
Генеральный директор, который не уважает решение собственника и остается в компании исключительно из паразитических соображений, жалок. И даже если его тактика сработает, даже если глупое решение, которое принял акционер, «как-нибудь само рассосется», все равно такой менеджер никогда не достигнет по-настоящему больших высот и результатов.

***
Где-то в мозгу у вас обязательно должна быть коробка передач. Надо только научиться ею пользоваться.

***
Есть только одна вещь, которая ценнее времени. Это твоя репутация. То, каким тебя запомнят после твоего ухода из компании, из профессии, из жизни. И на эту главную ценность нельзя жалеть ни времени, ни денег."""

In [3]:
DATA = TEXT.split(sep='\n\n***\n')

In [4]:
DATA

['У топ-менеджера две основные функции. Первая — добиваться поставленных целей. Вторая — научить коллег добиваться поставленных целей.',
 'Мы не любим ставить перед собой большие цели. Наш мозг, все наше естество противится этому. Почему? Ответ парадоксален, но точен. Потому что боимся, что они будут достигнуты! Да, да, именно поэтому.',
 'Какая у человека будет цель, такая у него будет и жизнь.',
 'Если у тебя нет стратегии — значит, у тебя нет бизнеса. Гендиректор, который не имеет цели хотя бы на пять лет, не может называть себя генеральным директором.',
 'Учите свою компанию учиться. Новые знания способны дать бизнесу больше, чем миллионы инвестиций.',
 'Люди любят, когда их хвалят. Для многих это важнее секса и денег. Потребность во внешнем признании сидит в нас так же глубоко, как волчья боязнь всего нового, что тоже неоднократно доказано психологическими экспериментами.',
 'Чего бы ты ни достиг, ты всегда находишься в точке возможной погибели',
 'Что бы там ни говорили про

In [5]:
import spacy
nlp = spacy.load('ru_core_news_md', disable=['tagger', 'attribute_ruler', 'senter', 'parser', 'ner'])


In [6]:
df = pd.DataFrame({'data':DATA})
data = df.data.apply(nlp)

def is_alphanum(token):
    return not token.is_punct \
        and not token.is_currency \
        and not token.is_digit \
        and not token.is_punct \
        and not token.is_oov \
        and not token.is_space \
        and not token.is_stop \
        and not token.like_num \
        and not token.pos_ == "PROPN"

data = data.to_frame() \
    .applymap(lambda x: ' '.join(t.lemma_ for t in x if is_alphanum(t))) \
    .data

data

0     топ менеджер основный функция добиваться поста...
1     любить ставить больший цель мозг естество прот...
2                                    человек цель жизнь
3     стратегия бизнес цель год называть генеральный...
4     компания учиться знание способный дать бизнес ...
5     любить хвалить многих важный секс деньга внешн...
6               ни достигнуть находишься точка погибель
7     ни говорить лихой время человек ценить честнос...
8     генеральный директор элемент косметика культур...
9     приходить работа делиться делиться делиться хо...
10    работа прийти устраиваться хорошенько нужный к...
11    изначально даваться черта характер склонность ...
12    задача человек выбрать человек хороший образ п...
13    человек важный эффективный подход помочь раскр...
14                                    покупатель прийти
15    требование ваш грубо противоречить ваш принцип...
16    говорить язык цифра цифра ответственность цифр...
17    директор уважать решение собственник остав

In [7]:
vect = CountVectorizer()

vect.fit(data)
print(vect.get_feature_names_out())

['акционер' 'бизнес' 'больший' 'босс' 'боязнь' 'бояться' 'важный' 'ваш'
 'вещь' 'влияние' 'внешний' 'вожак' 'волчий' 'воля' 'время' 'встретиться'
 'выбирать' 'выбрать' 'выполнять' 'высота' 'ген' 'генеральный' 'главный'
 'глубоко' 'глупый' 'говорить' 'год' 'грубо' 'даваться' 'дать' 'делать'
 'делиться' 'деньга' 'диктоваться' 'директор' 'добиваться' 'доказать'
 'достигнуть' 'естество' 'жалеть' 'жалок' 'жизненный' 'жизнь' 'задача'
 'занятие' 'запомнить' 'заражать' 'знание' 'изначально' 'инстинкт'
 'исключительно' 'какой' 'карьерист' 'клиент' 'коллега' 'компания'
 'коробка' 'косметика' 'культура' 'лихой' 'любить' 'менеджер' 'миллион'
 'минимум' 'многих' 'мозг' 'набор' 'называть' 'настоящему' 'настроение'
 'научить' 'научиться' 'находишься' 'неизбежно' 'неоднократно' 'ни'
 'новый' 'нужный' 'обещание' 'образ' 'обстоятельство' 'обязательно'
 'определённый' 'основный' 'оставаться' 'остальное' 'ответственность'
 'парадоксальный' 'паразитический' 'передать' 'передача' 'погибель'
 'подход' 'подхо

In [8]:
sorting = np.argsort(vect.transform(data).toarray().sum(axis=0))[::-1]
print("Most popular words CountVectoriser")
vect.get_feature_names_out()[sorting[:20]]

Most popular words CountVectoriser


array(['человек', 'компания', 'ни', 'важный', 'директор', 'время',
       'достигнуть', 'цель', 'ценность', 'настроение', 'цифра',
       'делиться', 'задача', 'любить', 'сотрудник', 'деньга', 'говорить',
       'менеджер', 'принцип', 'добиваться'], dtype=object)

In [9]:
data

0     топ менеджер основный функция добиваться поста...
1     любить ставить больший цель мозг естество прот...
2                                    человек цель жизнь
3     стратегия бизнес цель год называть генеральный...
4     компания учиться знание способный дать бизнес ...
5     любить хвалить многих важный секс деньга внешн...
6               ни достигнуть находишься точка погибель
7     ни говорить лихой время человек ценить честнос...
8     генеральный директор элемент косметика культур...
9     приходить работа делиться делиться делиться хо...
10    работа прийти устраиваться хорошенько нужный к...
11    изначально даваться черта характер склонность ...
12    задача человек выбрать человек хороший образ п...
13    человек важный эффективный подход помочь раскр...
14                                    покупатель прийти
15    требование ваш грубо противоречить ваш принцип...
16    говорить язык цифра цифра ответственность цифр...
17    директор уважать решение собственник остав

In [10]:
vect = TfidfVectorizer(norm=None)

vect.fit(data)
print(vect.get_feature_names_out())

['акционер' 'бизнес' 'больший' 'босс' 'боязнь' 'бояться' 'важный' 'ваш'
 'вещь' 'влияние' 'внешний' 'вожак' 'волчий' 'воля' 'время' 'встретиться'
 'выбирать' 'выбрать' 'выполнять' 'высота' 'ген' 'генеральный' 'главный'
 'глубоко' 'глупый' 'говорить' 'год' 'грубо' 'даваться' 'дать' 'делать'
 'делиться' 'деньга' 'диктоваться' 'директор' 'добиваться' 'доказать'
 'достигнуть' 'естество' 'жалеть' 'жалок' 'жизненный' 'жизнь' 'задача'
 'занятие' 'запомнить' 'заражать' 'знание' 'изначально' 'инстинкт'
 'исключительно' 'какой' 'карьерист' 'клиент' 'коллега' 'компания'
 'коробка' 'косметика' 'культура' 'лихой' 'любить' 'менеджер' 'миллион'
 'минимум' 'многих' 'мозг' 'набор' 'называть' 'настоящему' 'настроение'
 'научить' 'научиться' 'находишься' 'неизбежно' 'неоднократно' 'ни'
 'новый' 'нужный' 'обещание' 'образ' 'обстоятельство' 'обязательно'
 'определённый' 'основный' 'оставаться' 'остальное' 'ответственность'
 'парадоксальный' 'паразитический' 'передать' 'передача' 'погибель'
 'подход' 'подхо

In [11]:
max_value = vect.transform(data).max(axis=0).toarray().ravel()
sorted_by_tfidf = max_value.argsort()
feature_names = np.array(vect.get_feature_names_out())

print("Признаки с наименьшими значениями tfidf")
print(feature_names[sorted_by_tfidf[:20]])
print("Признаки с наибольшими значениями tfidf")
print(feature_names[sorted_by_tfidf[-20:]])

Признаки с наименьшими значениями tfidf
['компания' 'цель' 'достигнуть' 'любить' 'важный' 'директор' 'мозг'
 'приходить' 'миллион' 'прийти' 'менеджер' 'бизнес' 'больший' 'говорить'
 'работа' 'хороший' 'генеральный' 'жизнь' 'сотрудник' 'деньга']
Признаки с наибольшими значениями tfidf
['минимум' 'косметика' 'язык' 'называть' 'многих' 'набор' 'человек' 'ни'
 'время' 'настроение' 'задача' 'ценность' 'клиент' 'ваш' 'добиваться'
 'поставить' 'принцип' 'решение' 'делиться' 'цифра']
